In [63]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

In [64]:
reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")




In [65]:
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
weights = baseline.calculate("household_weight", period=2024)



In [66]:
baseline_aca_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=2026).sum()
baseline_aca_eligible/1e6

40.35911041608722

In [67]:
baseline_aca_enrollment = baseline.calculate("takes_up_aca_if_eligible", map_to="person", period=2026).sum()
baseline_aca_enrollment/1e6

227.28695912942868

In [68]:
period = 2025
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


184,751,527 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [69]:
period = 2026

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up_r = reformed.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc_r  = reformed.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt_r = reformed.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up_r == 1) & (aca_ptc_r > 0)

people_with_ptc_takeup_wtd_r = (mask.astype(float) * person_wt_r).sum()

print(f"{people_with_ptc_takeup_wtd_r:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


23,752,636 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [70]:
period = 2026
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


18,379,449 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [71]:
year = 2026
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
aca_baseline = baseline.calculate("aca_ptc", map_to="household", period=year)
rating_area = baseline.calculate("slcsp_rating_area", map_to="household", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)
aca_reform = reformed.calculate("aca_ptc", map_to="household", period=year)

In [72]:
# Create a DataFrame with the outputs
data = {
    "household_id": household_id,
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "Self_Employment_Income": self_employment_income,
    "aca_baseline": aca_baseline,
    "aca_reform": aca_reform,
    }

df_outputs = pd.DataFrame(data)
df_outputs[df_outputs['household_id'] == 103176]


,household_id,State,Married,Num_Dependents,Employment_Income,Self_Employment_Income,aca_baseline,aca_reform
23890,103176,PA,1.0,3.0,94992.562805,91.854012,0.0,39058.191406


In [73]:
# -------------------------------------------------------------
# 0️⃣  Make sure the CPS household weight is in the DataFrame
# -------------------------------------------------------------
# If you already stuffed it in earlier, skip this.
df_outputs["weight"] = aca_baseline.weights    # aligns by household_id

# -------------------------------------------------------------
# 1️⃣  Define a weight threshold for “reasonably representative”
# -------------------------------------------------------------
MIN_WT = 10_000          # ↖ try 5_000 if you want a looser cut

df_big = df_outputs[df_outputs["weight"] >= MIN_WT].copy()

# -------------------------------------------------------------
# 2️⃣  Net PTC change and (optionally) weighted national impact
# -------------------------------------------------------------
df_big["net_change"] = df_big["aca_reform"] - df_big["aca_baseline"]
df_big["wt_change"]  = df_big["net_change"] * df_big["weight"]  # national $ impact

# -------------------------------------------------------------
# 3️⃣  Biggest ↑ increases and ↓ decreases, LIMITED to big-weight HHs
# -------------------------------------------------------------
N = 10   # how many households to show in each direction

cols = ["household_id", "State", "weight", "net_change", "wt_change"]

top_increases = df_big.nlargest(N, "net_change")[cols]
top_decreases = df_big.nsmallest(N, "net_change")[cols]

print("Most positive net-income changes (PTC boosts):")
display(top_increases)

print("\nMost negative net-income changes (PTC cuts):")
display(top_decreases)


Most positive net-income changes (PTC boosts):


,household_id,State,weight,net_change,wt_change
15170,63406,TX,28846.123047,20097.220703,5.797269e+08
11774,47863,FL,43584.601562,17123.369141,7.463152e+08
14377,60850,TX,46835.007812,15904.038086,7.448657e+08
14628,61712,TX,33571.246094,14010.416992,4.703472e+08
6960,25327,MO,12508.783203,13525.498047,1.691875e+08
8987,38686,NC,15900.470703,12249.720703,1.947763e+08
31780,135336,FL,95542.593750,12184.659180,1.164154e+09
19620,83988,CA,12765.500000,10092.767578,1.288392e+08
36863,159723,CO,16615.960938,7437.457031,1.235805e+08
4013,16074,OH,16507.593750,6990.998047,1.154046e+08



Most negative net-income changes (PTC cuts):


,household_id,State,weight,net_change,wt_change
15,99,ME,12122.632812,0.0,0.0
24,188,ME,14602.563477,0.0,0.0
27,206,ME,13875.028320,0.0,0.0
30,261,ME,25312.886719,0.0,0.0
31,275,ME,19168.126953,0.0,0.0
32,284,ME,30920.968750,0.0,0.0
35,315,ME,13098.019531,0.0,0.0
41,339,ME,18794.173828,0.0,0.0
44,356,ME,36464.535156,0.0,0.0
47,380,ME,43613.914062,0.0,0.0


In [74]:
df_outputs[df_outputs['household_id'] == 83988]


,household_id,State,Married,Num_Dependents,Employment_Income,Self_Employment_Income,aca_baseline,aca_reform,weight
19620,83988,CA,1.0,2.0,160877.21875,0.0,0.0,10092.767578,12765.5


In [75]:
# 0. Make sure net_change exists
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# 1. Flag households with any change
mask = df_outputs["net_change"] != 0          # True for ↑ or ↓

# 2. Weighted mean among those households
avg_net_change = (
    (df_outputs.loc[mask, "net_change"] * df_outputs.loc[mask, "weight"]).sum()
    / df_outputs.loc[mask, "weight"].sum()
)

print(f"Average weighted PTC change among households with any change: "
      f"${avg_net_change:,.2f}")


Average weighted PTC change among households with any change: $2,264.49


In [76]:
# ------------------------------------------------------------------
# 0.  Ensure supporting columns exist
# ------------------------------------------------------------------
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# ------------------------------------------------------------------
# 1.  Keep only households with a PTC in *both* scenarios
# ------------------------------------------------------------------
mask_both_ptc = (df_outputs["aca_baseline"] > 0) & (df_outputs["aca_reform"] > 0)
df_dual_ptc   = df_outputs[mask_both_ptc]

# ------------------------------------------------------------------
# 2.  Weighted average of the net change (household perspective)
# ------------------------------------------------------------------
avg_net_change_dual_hh = (
    (df_dual_ptc["net_change"] * df_dual_ptc["weight"]).sum()
    / df_dual_ptc["weight"].sum()
)

print(f"Average weighted PTC change among households with a PTC in both "
      f"baseline and reform: ${avg_net_change_dual_hh:,.2f}")


Average weighted PTC change among households with a PTC in both baseline and reform: $1,654.84


In [92]:
import numpy as np
from policyengine_us import Simulation

# -------------------------------
# 1. Pull household-level results
# -------------------------------
# ACA PTC (baseline and reform)
ptc_base   = baseline.calculate("aca_ptc", map_to="household", period=2026)
ptc_reform = reformed.calculate("aca_ptc",   map_to="household", period=2026)

# Household weights (same for both sims)
hh_wt      = baseline.calculate("household_weight", map_to="household", period=2026)

# -------------------------------
# 2. Weighted sum of the change
# -------------------------------
weighted_total_change = ptc_reform - ptc_base

# Optional: average change per household
weighted_total_change.sum()/1e9

30.206960672172997

In [104]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ------------------------------------------------------------------
# Brand hex codes (one-to-one with style.colors)
# ------------------------------------------------------------------
COLOR_BLUE               = "#2C6496"   # style.colors.BLUE  / BLUE_PRIMARY
COLOR_BLUE_LIGHT         = "#D8E6F3"   # style.colors.BLUE_LIGHT / BLUE_95
COLOR_LIGHT_GRAY         = "#F2F2F2"   # style.colors.LIGHT_GRAY
COLOR_MEDIUM_LIGHT_GRAY  = "#BDBDBD"   # style.colors.MEDIUM_LIGHT_GRAY
COLOR_DARK_GRAY          = "#616161"   # style.colors.DARK_GRAY

# ––– choose colours for positive vs. negative average bars –––
POS_COLOR = COLOR_BLUE
NEG_COLOR = COLOR_DARK_GRAY

# ------------------------------------------------------------------
# 1.  Pull baseline / reform net income + weights
# ------------------------------------------------------------------
net_base   = baseline.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
net_reform = reformed.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
weights    = baseline.calculate(
    "household_weight", map_to="household", period=2026
)

df = pd.DataFrame({
    "net_base": net_base,
    "delta":    net_reform - net_base,
    "weight":   weights,
})

# ------------------------------------------------------------------
# 2.  Weighted decile edges (baseline ranking)
# ------------------------------------------------------------------
def wquantile(values, qs, w):
    srt = np.argsort(values)
    values, w = values[srt], w[srt]
    cum_w = np.cumsum(w) / np.sum(w)
    return np.interp(qs, cum_w, values)

edges = wquantile(df["net_base"].values,
                  np.linspace(0, 1, 11), df["weight"].values)

df["decile"] = pd.cut(df["net_base"],
                      bins=edges,
                      labels=np.arange(1, 11),
                      include_lowest=True)

# ------------------------------------------------------------------
# 3.  Weighted average Δnet-income by decile
# ------------------------------------------------------------------
decile_avg = (
    df.groupby("decile")
      .apply(lambda g: np.average(g["delta"], weights=g["weight"]))
      .reset_index(name="avg_change")
)

# ------------------------------------------------------------------
# 4.  Use brand colours: blue if gain, dark-gray if loss
# ------------------------------------------------------------------
bar_colors = [
    POS_COLOR if v >= 0 else NEG_COLOR
    for v in decile_avg["avg_change"]
]

# ------------------------------------------------------------------
# 5.  Plot
# ------------------------------------------------------------------
fig = go.Figure(
    data=[
        go.Bar(
            x=decile_avg["decile"].astype(int),
            y=decile_avg["avg_change"],
            marker_color=bar_colors,
            text=decile_avg["avg_change"].apply(lambda v: f"${v:,.0f}"),
            textposition="inside",
        )
    ],
    layout=dict(
        title="Impact of Extending IRA PTC Expansion by Income Decile – 2026",
        xaxis_title="Income Decile",
        yaxis_title="Average change in household net income ($)",
        showlegend=False,
    )
)
fig.add_hline(y=0, line_width=1, line_color="black")
fig.show()
fig.update_xaxes(dtick=1)        # show 1-10 instead of only the evens
